# 0. Import

In [ ]:
! pip install datasets
! pip install transformers
! pip install sentencepiece

In [ ]:
import pandas as pd
from datasets import *
from transformers import BertTokenizerFast, DistilBertTokenizerFast, BertConfig, DistilBertConfig, BertForMaskedLM, DistilBertForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from tokenizers import *
import torch
import os
import json

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# 1. Clean Korean Legal Corpus(CKLC)

## 1-1. Law data

In [ ]:
df = pd.read_csv('./data/cases(20220521).csv', index_col=False)  # index_col = 'case_number
df = df.dropna(how='any')
print(df.isnull().sum())
print(">> Number of Korea legal precedents : ", len(df))

case_name            0
case_number          0
date                 0
case_code            0
judgment_issue       0
judgment_summary     0
judgment_contents    0
dtype: int64
>> Number of Korea legal precedents :  62919


In [ ]:
df.head()

,case_name,case_number,date,case_code,judgment_issue,judgment_summary,judgment_contents
0,미성년자의제강간,2021노824,20220415.0,400102.0,피고인이 카카오톡 오픈채팅방을 통해 미성년자 여 11세 을 알게 된 후 당시 13세...,피고인이 카카오톡 오픈채팅방을 통해 미성년자 여 11세 을 알게 된 후 당시 13세...,【피 고 인】 피고인【항 소 인】 쌍방【검 사】 이지은 외 1인【변 호 인】 ...
2,손해배상(기),2021나24173,20220323.0,400101.0,이 을 상대로 소송을 제기하자 이 과 소송위임계약을 체결하여 이 을 대리하여 소송을...,이 을 상대로 소송을 제기하자 이 과 소송위임계약을 체결하여 이 을 대리하여 소송을...,"【원고, 항소인 겸 피항소인】 원고 (소송대리인 법무법인 우정 담당변호사 김병구 외..."
4,손해배상(의)[환자가 치료 도중 뇌출혈로 사망하자 의료과실에 의한 손해배상을 구하는...,2018다263434,20220317.0,400101.0,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 특히 환자가 병원에서...,의사가 진찰 치료 등의 의료행위를 할 때에는 사람의 생명 신체 건강을 관리하는 업무...,"【원고, 상고인】 원고 1 외 2인 (소송대리인 변호사 신현호 외 4인)【피고, 피..."
5,상표법위반·업무상배임[타인의 상표가 부착된 제품을 무상으로 제공한 경우 상표법위반죄...,2021도2180,20220317.0,400102.0,1 상표법상 상표의 사용 및 상품의 의미 2 피고인 은 상표권자의 허락 없이 상표를...,1 상표법상 상표의 사용이란 상품 또는 상품의 포장에 상표를 표시하는 행위 상품 또...,【피 고 인】 피고인 1 외 1인【상 고 인】 검사【원심판결】 서울서부지법 2021...
6,소유권이전등기·소유권이전등기[피고로부터 부동산을 증여받은 원고들이 피고를 상대로 증...,"2017다207475, 207482",20220311.0,400101.0,1 수증자의 범죄행위를 원인으로 한 증여계약의 해제를 규정하고 있는 민법 제556조...,1 민법 제556조 제1항 제1호는 수증자가 증여자에 대하여 증여자 또는 그 배우자...,"【원고(반소피고), 상고인】 원고(반소피고) (소송대리인 법무법인(유한) 세종 담당..."


In [ ]:
# Text 저장
df['judgment_summary'].to_csv('law_summary.txt', index=False)
df['judgment_issue'].to_csv('law_issue.txt', index=False)
df['judgment_contents'].to_csv('law_contents.txt', index=False)

## 1-2. Data Sum

In [ ]:
# Data_Sum
files = [
         'law_summary.txt',
         'law_issue.txt',
         'law_contents.txt',
         'contract.txt'
         ]
dataset = load_dataset("text", data_files=files, split="train")

data = dataset.train_test_split(test_size=0.08)

Using custom data configuration default-4979e5acb8be5700


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-4979e5acb8be5700/0.0.0/acc32f2f2ef863c93c2f30c52f7df6cc9053a1c2230b8d7da0d210404683ca08. Subsequent calls will reuse this data.


In [ ]:
# Text Data check!
for t in data['train']['text'][:10]:
    print(t)
    print("===="*50)

"【피 고 인】 【상 고 인】   피고인들【변 호 인】   변호사 박천식(피고인들에 대하여)【원심판결】 서울형사지방법원 1986.11.28 선고 86노4551 판결【주    문】  상고를 모두 기각한다.【이    유】  변호인의 상고이유를 본다.  상고논지는 피고인들은 이 사건 물건들이 장물인 정을 모르고 취득하였다는데 있는 바, 원래         장물취득죄의 주관적 요건인 장물이라는 정의 인식은 장물성에 관한 미필적 인식이 있으면 충분하다할 것인데 원심이 유지한 제1심판결이 들고 있는 증거들에 의하면 피고인들이 이 사건 물건들을 각 취득함에 있어 적어도 그 물건들의 장물성에 관한 미필적 인식은 가지고 있었음을 충분히 인정할 수 있어 원심이 피고인들에 대한 판시 각 범죄사실을 인정하여 장물취득죄로 의율한 제1심판결을 지지한 조치를 정당한 것으로 수긍할 수 있고, 거기에 채증법칙을 위배하여 사실을 오인하고 장물취득에 관한 법리를 오해한 위법있다 할 수 없다. 논지는 이유없다.        그러므로 상고를 모두 기각하기로 관여법관의 일치된 의견으로 주문과 같이 판결한다.대법관 황선당(재판장) 이병후 김달식"
갑 은 보안상 필요한 때에는 다음 사항 등을 을 에게 의뢰할 수 있으며 이에 순응해야한다
상법 제679조의 추정은 보험목적의 양수인에게 보험승계가 없다는 것이 증명된 경우에는 번복된다고 할 것인데 보험목적의 양수인이 그 보험목적에 대한 1차 보험계약과 피보험이익이 동일한 보험계약을 체결한 사안에서 제1차 보험계약에 따른 보험금청구권에 질권이 설정되어 있어 보험사고가 발생할 경우에도 보험금이 그 질권자에게 귀속될 가능성이 많아 1차보험을 승계할 이익이 거의 없고 또한 그 양수인이 그 보험목적에 관하여 손해의 전부를 지급받을 수 있는 필요충분한 보험계약을 체결한 경우 양수인에게는 보험승계의 의사가 없었다고 봄이 상당하고 따라서 1차보험은 양수인에게 승계되지 아니하였으므로 양수인이 체결한 보험이 중복보험에 해당하지 않는다
"【원고,상고인】 【피고,피상고인】   유

# 2. Training the Tokenizer(Make Vocab)
    - Byte-Pair Encoding(BPE) : Roberta, GPT-2
    - WordPiece : BERT, DistilBERT
    - SentencePiece : ALBERT, XLNet, T5

In [ ]:
def dataset_to_text(dataset, output_filename='data.txt'):
    with open(output_filename, "w") as f:
        for t in dataset['text']:
            print(t, file=f)

# Save train / text dataset -> txt 
dataset_to_text(data["train"], "train_3.txt")
dataset_to_text(data["test"], "test_3.txt")

In [ ]:
# Parameters
special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"]
files = ["train_3.txt", "test_3.txt"]
vocab_size = 30522
max_length = 512
truncate_longer_samples = False  # No cut!

In [ ]:
# Initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()
# Train the tokenizer
tokenizer.train(
    files=files,
    vocab_size=vocab_size,
    special_tokens=special_tokens)
# Enable truncation up to the maximum "512 tokens"
tokenizer.enable_truncation(max_length=max_length)

In [ ]:
model_path = "pretrained-KorLawBERT_2"
# Make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)

# Save the tokenizer in model_path
tokenizer.save_model(model_path)

In [ ]:
# Dumping some of the tokenizer config to config file, including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

In [ ]:
# When the tokenizer is trained and configured, load it as BertTokenizerFast
model_path = "pretrained-KorLawBERT_2"
tokenizer = BertTokenizerFast.from_pretrained(model_path)

# 3. Tokenizing the Dataset

In [ ]:
def encode_with_truncation(examples):
    """Mapping function to tokenize the sentences passed with truncation"""
    return tokenizer(examples["text"],
                     truncation=True,
                     padding="max_length",
                     max_length=max_length,
                     return_special_tokens_mask=True)

def encode_without_truncation(examples):
  """Mapping function to tokenize the sentences passed without truncation"""
  return tokenizer(examples["text"], return_special_tokens_mask=True)


# 1.Encoding : The encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

# 2. Tokenizing the Train & Test Dataset 
train_dataset = data['train'].map(encode, batched=True)
test_dataset = data['test'].map(encode, batched=True)

if truncate_longer_samples:
  # remove other columns and set input_ids and attention_mask as 
  train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
  test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
  test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
  train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

#train_dataset, test_dataset

In [ ]:
# 3. Main data processing function that will concatenate all texts from our dataset and generate chunks of max_seq_length.

def group_texts(examples):
    # 1.Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # 2. Drop the small remainder
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # 3. Split by Chunk of Max_len
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

# 'batched=True' : This map processes 1,000 Texts together, so group_texts THROWS AWAY a remainder for each of those groups of 1,000 texts. 
if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True, batch_size=2000,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts,  batched=True, batch_size=2000,
                                  desc=f"Grouping texts in chunks of {max_length}")

Grouping texts in chunks of 512:   0%|          | 0/126 [00:00<?, ?ba/s]

Grouping texts in chunks of 512:   0%|          | 0/11 [00:00<?, ?ba/s]

# 4. Loading the Model

In [ ]:
# 1. initialize the model with the config
model_config = BertConfig(vocab_size=vocab_size)

# 2. BertForMasked
model = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-4000"))
#model = BertForMaskedLM(config=model_config)

# 5. Pre-Training(BERT MLM Task)

In [ ]:
# MLM : Randomly Masking 20% of the tokens For the MLM Task
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=True,
                                                mlm_probability=0.2)
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,            # number of training epochs
    per_device_train_batch_size=10, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=1000,              # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

In [ ]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train the model(110,000 Steps)
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 141550
  Num Epochs = 10
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 80
  Gradient Accumulation steps = 8
  Total optimization steps = 17690


Step,Training Loss,Validation Loss
1000,7.199300,7.147852
2000,7.134300,7.094993
3000,7.079900,7.049032
4000,7.036900,7.015871


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 12324
  Batch size = 64
Saving model checkpoint to pretrained-KorLawBERT_2/checkpoint-1000
Configuration saved in pretrained-KorLawBERT_2/checkpoint-1000/config.json
Model weights saved in pretrained-KorLawBERT_2/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 12324
  Batch size = 64
Saving model checkpoint to pretrained-KorLawBERT_2/checkpoint-2000
Configuration saved in pretrained-KorLawBERT_2

Step,Training Loss,Validation Loss
1000,7.199300,7.147852
2000,7.134300,7.094993
3000,7.079900,7.049032
4000,7.036900,7.015871
5000,6.995200,6.987613
6000,6.968300,6.962043
7000,6.940400,6.936453
8000,6.915300,6.916150
9000,6.898500,6.899807
10000,6.877500,6.887833


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 12324
  Batch size = 64
Saving model checkpoint to pretrained-KorLawBERT_2/checkpoint-5000
Configuration saved in pretrained-KorLawBERT_2/checkpoint-5000/config.json
Model weights saved in pretrained-KorLawBERT_2/checkpoint-5000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 12324
  Batch size = 64
Saving model checkpoint to pretrained-KorLawBERT_2/checkpoint-6000
Configuration saved in pretrained-KorLawBERT_2